In [1]:
# Set up the mount of the google drive.
!pip install librosa

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd
import wave
import sys
import os
import librosa
import librosa.display
import xgboost as xgb
from  sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import sklearn.naive_bayes as nb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier
from sklearn.neighbors import kd_tree
import seaborn as sn
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.kd_tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packag

In [4]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint


from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

Using TensorFlow backend.


In [0]:
# Load wav data.
import pickle
filename = '/content/drive/My Drive/CSCE 666/Project/Feature/w_c_dataset.pickle'
infile = open(filename,'rb')
[sound, sr, lengths, times, wc_label] = pickle.load(infile)
infile.close()

In [0]:
# Obtain wav file ID.
import glob
wav_list = [f.split('.wav')[0] for f in glob.glob("/content/drive/My Drive/CSCE 666/Project/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/*.wav")]
wav_list.sort()

wav_ids = []
wav_names = []
for wav_path in wav_list:
  ind = wav_path.rfind("/")
  wav_name = wav_path[ind+1:]
  Id = int(wav_name[:wav_name.find("_")])
  wav_ids.append(Id)
  wav_names.append(wav_name)

In [0]:
import pandas as pd
demographic = pd.read_csv('/content/drive/My Drive/CSCE 666/Project/Feature/wav_info_COPD_missdata_samplemean.csv', header=None)
demographic_data = demographic.values
wav_disease_labels = demographic_data[:, -1]
demographic_data = demographic_data[0:, 1:-1] # Remove the id column (the first column)

In [0]:
"""
# 8 disease classification
def updatelabel(labels): 
    l=[]
    for label in labels:
      if label == 0:
        a = np.array([1, 0, 0, 0, 0, 0, 0, 0])
      elif label == 1:
        a = np.array([0, 1, 0, 0, 0, 0, 0, 0])
      elif label == 2:
        a = np.array([0, 0, 1, 0, 0, 0, 0, 0])
      elif label == 3:
        a = np.array([0, 0, 0, 1, 0, 0, 0, 0])  
      elif label == 4:
        a = np.array([0, 0, 0, 0, 1, 0, 0, 0])  
      elif label == 5:
        a = np.array([0, 0, 0, 0, 0, 1, 0, 0])
      elif label == 6:
        a = np.array([0, 0, 0, 0, 0, 0, 1, 0])
      else:
        a = np.array([0, 0, 0, 0, 0, 0, 0, 1])
      l.append(a)
    return l
  """
# COPD vs. non-COPD
def updatelabel(labels):
    l=[]
    for label in labels:
      if label == 0:
        a = np.array([1, 0])
      if label == 1:
        a = np.array([0, 1]) 
      l.append(a)
    return l

In [9]:
new_wav_disease_labels = updatelabel(wav_disease_labels)
print(len(new_wav_disease_labels))
print(new_wav_disease_labels)

920
[array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 0]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array

In [0]:
print(type(demographic_data))
print(demographic_data.shape)
print(demographic_data[0])
print(demographic_data[1])
print(demographic_data[2])
print(demographic_data[3])
print(demographic_data[4])
print(demographic_data[0].shape)
#print(demographic_data[0:, 1:])
print(len(sound))

In [0]:
import librosa
def find_mfcc_max_length(sound, sr):
  num_sound = len(sound)
  max_length = 0
  for i in range(num_sound):
    mfccs = librosa.feature.mfcc(y=sound[i], sr=sr[i], n_mfcc=40)
    mfcc_length = mfccs.shape[1]
    if mfcc_length > max_length:
      max_length = mfcc_length

  return max_length

In [0]:
max_len = find_mfcc_max_length(sound, sr)
print(max_len)

3704


In [0]:
# Define feature extraction function: MFCC + demographic info
import librosa
max_len = 3704
def extract_feature(X, sample_rate):
    mfccs = librosa.feature.mfcc(y=X, sr=sample_rate,n_mfcc=40)
    
    # Pad  mfcc till the end
    pad_width_mfcc = max_len - mfccs.shape[1]
    
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width_mfcc)), mode='constant')
    
    return mfccs
    # Make mfccs as a vector
    #feat = mfccs[0, :]
    #for i in range(39):
    #  feat = np.hstack((feat, mfccs[i+1, :]))
    #print(feat.shape)
    #return feat
    

In [0]:
# Extract features
featset = []
for i in range(len(sound)):
    a = extract_feature(sound[i], sr[i])
    featset.append(a)


In [12]:
featset = np.array(featset)
featset.shape

(920, 40, 3704)

In [0]:
# add channel dimension for CNN
featset = np.reshape(featset, (*featset.shape,1))

new_wav_disease_labels=np.asarray(new_wav_disease_labels)
a=np.zeros(new_wav_disease_labels.shape[0])
for i in range(new_wav_disease_labels.shape[0]):
    for j in range(new_wav_disease_labels.shape[1]):
        if new_wav_disease_labels[i][j]==1:
            a[i]=j

# One-hot encode labels
le = LabelEncoder()
i_labels = le.fit_transform(a)
oh_labels = to_categorical(i_labels) 

In [14]:
featset.shape

(920, 40, 3704, 1)

In [0]:
# train test split
x_train, x_test, y_train, y_test = train_test_split(featset, oh_labels, stratify=new_wav_disease_labels, 
                                                    test_size=0.2, random_state = 42)

In [0]:
num_rows = 40
num_columns = 3704
num_channels = 1

num_labels = 2 # COPD vs. non-COPD
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=filter_size,
                 input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(GlobalAveragePooling2D())

model.add(Dense(32, activation='relu')) 

model.add(Dense(num_labels, activation='softmax')) 

In [0]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [18]:
# Display model architecture summary 
model.summary()

# train model
num_epochs = 250
num_batch_size = 128



callbacks = [
    ModelCheckpoint(
        filepath='/content/drive/My Drive/CSCE 666/Project/Eric\'s Notebooks/saved_models/MFCC_CNN_Binary_{epoch:02d}.h5', 
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_accuracy` score has improved.
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1)
]
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, verbose=1,
          validation_data=(x_test, y_test), callbacks=callbacks)


duration = datetime.now() - start
print("Training completed in time: ", duration)

#model.fit(x_train, y_train,
#          batch_size=batch_size,
#          epochs=epochs,
#          verbose=1,
#          validation_data=(x_test, y_test),
#          callbacks=[history])

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 39, 3703, 16)      80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 1851, 16)      0         
_________________________________________________________________
dropout (Dropout)            (None, 19, 1851, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 18, 1850, 32)      2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 925, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 925, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 924, 64)        8

In [19]:
model_predictions = model.predict(x_test) 
model_labels = model_predictions
#print(model_predictions[5])
for i in range(model_predictions.shape[0]):
  if model_predictions[i][0] > 0.5:
    model_labels[i][0] = 1
    model_labels[i][1] = 0
  else:
    model_labels[i][0] = 0
    model_labels[i][1] = 1

#print(model_labels[5])
# print classification report 
print(classification_report(y_test, model_predictions))
print(accuracy_score(y_test,model_predictions))

              precision    recall  f1-score   support

           0       0.80      0.48      0.60        25
           1       0.92      0.98      0.95       159

   micro avg       0.91      0.91      0.91       184
   macro avg       0.86      0.73      0.78       184
weighted avg       0.91      0.91      0.90       184
 samples avg       0.91      0.91      0.91       184

0.9130434782608695
